In [7]:
!pip install transformers -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install torch -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install pandas -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install scikit_learn -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [8]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader
from transformers import AdamW
import pandas as pd
import os
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tqdm import tqdm
import json
from collections import OrderedDict
from datetime import datetime
import logging
logging.basicConfig(level=logging.INFO, format='%(message)s')

In [9]:
# huggingface 分词器，wordpiece
tokenizer = BertTokenizer.from_pretrained("kykim/bert-kor-base")

In [10]:
### 读入数据
def read_data(file):
    texts = []
    labels = []
    data = pd.read_excel(file, engine='openpyxl')
    for row in data.itertuples():
        label = getattr(row, 'label')
        review = str(getattr(row, 'review'))[1:-1]
        texts.append(review)
        labels.append(label)
    assert len(texts) == len(labels)
    return texts, labels

In [11]:
texts, labels = read_data('Ko.xlsx')

In [12]:
len(texts)

30000

In [13]:
###  训练集和验证集
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=43, stratify=labels)

In [14]:
len(train_labels),len(val_labels),len(train_texts),len(val_texts)

(24000, 6000, 24000, 6000)

In [15]:
train_texts[:10]

['제 엘리트 체육은 지양할때다 저개 뭐냐??개판이',
 '번건 잘했다고 본',
 '관중 경기 아니였나요?',
 'c에서 박석민외3명에게 구상권 신청할듯네명은 거지될',
 '여자 쌍둥이 자매를 며느리삼고싶다했다는데... 에휴 그만합시다 이러다 김연경선수 갈굴까 걱정됩니',
 '포츠와 정치를 연관짓는다고 한국정치인 비판하는 일본놈들.... 양심있으면 반성하',
 '자호구 없애라. 경기 더럽게 재미없다. 몸통 펑!!하고 소리나도 점수 안오르고 진짜 뭐냐? 예전처럼 심판 몇명이서 스위치 눌러',
 '본 올림픽 부정적으로 보다만 하다가 중계는 뭔 중계냐? 대한민국의 언론수준과 이에 놀아나는 생각없는 국민들...참으로 가볍고 가볍워...ㅜ ',
 '능하면 정권을 탐하지 말아야지----- 4년 내내 상대를 까다가 ---- 올림픽 놀러가려고 대접해달면----- 니라면 하겠냐-----죽창이 뭐냐---- 조선시대냐? 공산당이냐????--- 북한넘들 지주들 죽창으로 찔러죽였던--',
 '도 방문하고 꼭 방일하기를..']

In [16]:
train_labels[:10]

[0, 1, 0, 0, 1, 0, 0, 0, 0, 1]

In [17]:
###  求最大长度，为后面分词做准备
max_len = max([len(item) for item in train_texts])
print(max_len)

max_len = max([len(item) for item in val_texts])
print(max_len)

298
298


In [18]:
### label和id进行映射
label2id = OrderedDict({item: idx for idx, item in enumerate(set(train_labels + val_labels))})
id2label = OrderedDict({v: k for k, v in label2id.items()})

In [19]:
#  训练集和验证集 分词
train_encodings = tokenizer(train_texts,
                            truncation=True,
                            padding=True,
                            max_length=128)
val_encodings = tokenizer(val_texts,
                          truncation=True,
                          padding=True,
                          max_length=128)

In [20]:
# PyTorch Dataset  创建Dataset
class CuDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        idx = int(idx)
        item = {
            key: torch.tensor(val[idx])
            for key, val in self.encodings.items()
        }
        item['labels'] = torch.tensor(label2id[self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [21]:
train_dataset = CuDataset(train_encodings, train_labels)
val_dataset = CuDataset(val_encodings, val_labels)

In [22]:
#  创建Dataloader
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

### 加载模型

In [23]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device(
    'cpu')  # 使用cpu或者gpu
model = BertForSequenceClassification.from_pretrained(
    "kykim/bert-kor-base", num_labels=len(label2id))  #来加载模型，那么线性层 + 激活函数的权重就会随机初始化。我们的目的，就是通过微调，学习到线性层 + 激活函数的权重。
model.to(device)
model.train()

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [24]:
### 计算Accuracy，Precision，Recall，F1 score，confusion_matrix，classification_report
def compute_metrics(labels, preds):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    print(f'accuracy: {accuracy}\n')
    print(f'precision: {precision}\n')
    print(f'recall: {recall}\n')
    print(f'f1: {f1}\n')
    print(confusion_matrix(labels, preds))
    print(classification_report(labels, preds))
    return f1

### 13.评估模型

In [25]:
@torch.no_grad()
def eval_model(model, eval_loader):
    model.eval()
    labels = []
    preds = []
    for idx, batch in enumerate(eval_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels.extend(batch['labels'].numpy())
        outputs = model(input_ids,attention_mask=attention_mask)  # 输出所有概率
        preds.extend(torch.argmax(outputs[0], dim=-1).cpu().numpy())  # 拿到标签
    f1 = compute_metrics(labels, preds)
    model.train()
    return f1

### 14.训练模型

In [26]:
optim = AdamW(model.parameters(), lr=1e-5)  # 声明优化器
step = 0
best_f1 = 0
epoch = 10
for epoch in range(epoch):
    for idx, batch in tqdm(enumerate(train_loader),
                           total=len(train_texts) // batch_size):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids=input_ids, labels=labels, attention_mask=attention_mask)
        loss = outputs[0]  # 计算Loss
        logging.info(
            f'Epoch-{epoch}, Step-{step}, Loss: {loss.cpu().detach().numpy()}')
        step += 1
        loss.backward()
        optim.step()

    print(f'Epoch {epoch}, start evaluating.')
    f1 = eval_model(model, eval_loader)  # 评估模型
    if f1 > best_f1:
        print(f'best_f1: {f1}')
        model.save_pretrained('model_best')  # 保存模型
        tokenizer.save_pretrained('model_best')
        best_f1 = f1

 26%|████████████████████▊                                                          | 99/375 [25:48<1:13:18, 15.94s/it]Epoch-0, Step-99, Loss: 0.3467811942100525


 53%|██████████████████████████████████████████▍                                     | 199/375 [51:57<45:39, 15.57s/it]Epoch-0, Step-199, Loss: 0.4218702018260956


 80%|██████████████████████████████████████████████████████████████▏               | 299/375 [1:17:41<19:50, 15.67s/it]Epoch-0, Step-299, Loss: 0.35907813906669617


100%|██████████████████████████████████████████████████████████████████████████████| 375/375 [1:37:06<00:00, 15.54s/it]


Epoch 0, start evaluating.
accuracy: 0.872

precision: 0.6913716814159292

recall: 0.5610412926391383

f1: 0.6194251734390487

[[4607  279]
 [ 489  625]]
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      4886
           1       0.69      0.56      0.62      1114

    accuracy                           0.87      6000
   macro avg       0.80      0.75      0.77      6000
weighted avg       0.86      0.87      0.87      6000

best_f1: 0.6194251734390487


...

Epoch-9, Step-3375, Loss: 0.06399337947368622
 26%|████████████████████▋                                                          | 98/375 [26:56<1:16:25, 16.55s/it]Epoch-9, Step-3473, Loss: 0.01864663138985634


 52%|█████████████████████████████████████████▊                                      | 196/375 [53:30<46:22, 15.54s/it]Epoch-9, Step-3571, Loss: 0.048235874623060226


 78%|█████████████████████████████████████████████████████████████▏                | 294/375 [1:18:56<20:51, 15.45s/it]Epoch-9, Step-3669, Loss: 0.023858746513724327


100%|██████████████████████████████████████████████████████████████████████████████| 375/375 [1:39:59<00:00, 16.00s/it]


Epoch 9, start evaluating.
accuracy: 0.8475

precision: 0.5893980233602875

recall: 0.5888689407540395

f1: 0.5891333632689717

[[4429  457]
 [ 458  656]]
              precision    recall  f1-score   support

           0       0.91      0.91      0.91      4886
           1       0.59      0.59      0.59      1114

    accuracy                           0.85      6000
   macro avg       0.75      0.75      0.75      6000
weighted avg       0.85      0.85      0.85      6000



In [27]:
def predict(model, tokenizer, text):
    encoding = tokenizer(text,
                         return_tensors="pt",
                         max_length=128,
                         truncation=True,
                         padding=True)
    encoding = {k:v.to(device) for  k,v in encoding.items()}
    outputs = model(**encoding)
    #pred = id2label[torch.argmax(outputs[0], dim=-1).numpy()[0]]
    pred = id2label[torch.argmax(outputs[0], dim=-1).cpu().detach().numpy()[0]]
    return pred

In [28]:
tokenizer = BertTokenizer.from_pretrained("model_best")
model = BertForSequenceClassification.from_pretrained(
    "model_best", num_labels=len(label2id))
model.to(device)    

In [ ]:
import numpy as np
result_dict = {}
for root, dirs, files in os.walk('test', topdown=True):
    for name in files:
        all_preds = []
        print(f'process file: {name}')
        try:
            with open(os.path.join(root, name), 'r', encoding='utf8') as f:
                lines = f.readlines()
        except:
            with open(os.path.join(root, name), 'r', encoding='gb18030') as f:
                lines = f.readlines()
        for idx, line in enumerate(lines):
            line = line.strip()
            if not line:
                continue
            text = line
            pred=predict(model,tokenizer,line)
            all_preds.append(pred)
        base_name_date = '-'.join([os.path.basename(name)[4:6], os.path.basename(name)[6:8]])
        result_dict[datetime.strptime(base_name_date, "%m-%d")] = [round(
            sum(all_preds) / len(all_preds), 2), len(all_preds)]
result_dict = sorted(result_dict.items(), key=lambda x: x[0])
np.save('result_dict_Ko.npy', result_dict)

process file: 20210623.csv
process file: 20210624.csv
process file: 20210625.csv
process file: 20210626.csv
process file: 20210627.csv
process file: 20210628.csv
process file: 20210629.csv
process file: 20210630.csv
process file: 20210701.csv
process file: 20210702.csv
process file: 20210703.csv
process file: 20210704.csv
process file: 20210705.csv
process file: 20210706.csv
process file: 20210707.csv
process file: 20210708.csv
process file: 20210709.csv
process file: 20210710.csv
process file: 20210711.csv
process file: 20210712.csv
process file: 20210713.csv
process file: 20210714.csv
process file: 20210715.csv
